In [1]:
import csv
import json
import sqlite3
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

In [39]:
def plot_learning_curve_single_subject(marks_conf,counterbalance):

    fig, ax = plt.subplots(figsize=(5, 3))
    rect = patches.Rectangle((0,-0.2), 3, 1.4, linewidth=1,  facecolor='yellow', alpha=0.5)#edgecolor='r',
    conf_mat = marks_conf[9]
    marks_acc = [(conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()]
    for i in range(1,6):
        conf_mat = marks_conf[(i+1)*8-1] - marks_conf[i*8-1]
        acc = (conf_mat.diagonal() / (conf_mat.sum(1)+1e-7)).mean()
        marks_acc.append(acc)

    ax.scatter(range(len(marks_acc)),marks_acc,c='black',s=60)

    for i in range(len(marks_acc)-1):
        ax.plot([i,i+1],[marks_acc[i],marks_acc[i+1]],c='black')
    plt.yticks([_*0.1 for _ in range(0,11,2)],[round(_*0.1,1) for _ in range(0,11,2)],fontsize=20)
    plt.xticks(range(len(marks_acc)),[_*8 for _ in range(1,len(marks_acc)+1)],fontsize=20)
    plt.suptitle('counterbalance: {}'.format(counterbalance), fontsize=20)
    # Add the patch to the Axes
    ax.add_patch(rect)
    ax.xaxis.set_ticks_position('none') 
    ax.yaxis.set_ticks_position('none') 
    ax.spines[['right', 'left']].set_visible(False)

def construct_confmat():
    labels = ['h_azure','h_histogram','h_google','h_chatgpt']#,'h_catch']
    confmatrices = {**{'h_machine':np.zeros((2,2))}, **{label: np.zeros((1,2)) for label in labels}}
    return confmatrices

def print_metadata(data,dummy_mark='NA'):
    print("====="*20)
    datetime_object1 = datetime.strptime(data[11].split('.')[0], '%Y-%m-%d %H:%M:%S')
    datetime_object2 = datetime.strptime(data[13].split('.')[0], '%Y-%m-%d %H:%M:%S')
    time_used = datetime_object2 - datetime_object1
    time_used = round(time_used.seconds/60,1)
    print("[Finished Date]",datetime_object2)
    print("[Time Used] {} mins".format(time_used))
    print("[Questionnaire]", json.loads(data[-1])["questiondata"])
    print("[Counterbalance]", json.loads(data[-1])["counterbalance"])
    print("[Dummy Test Acc]", dummy_mark)
    return time_used, json.loads(data[-1])["counterbalance"]

def get_gt_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'a':'Human','b':'Machine','c':'Machine','d':'Machine','e':'Machine'}[pagename.split('_')[1]]

def get_model_from_pagename(pagename):
    # example:  'page_name': '568_e_0.html/',
    return {'b':'azure','c':'histogram','d':'google','e':'chatgpt'}[pagename.split('_')[1]]

conn = sqlite3.connect("ec2_db/turing_color_const.db")
cursor = conn.cursor()
cursor.execute("select * from tablechairDB;")

label_dict = {'H':0,'AI':1,'human':0,'machine':1,'Human':0,'Machine':1,'azure':1,'histogram':1,'google':1,'chatgpt':1}
oneWord_curr_d = {'0':True,'1':False} # dummy test gt
topic_d = {'1':True,'2':False} # user response to dummy tests
modelname_dict = {'catch':'CatchTrial','azure':'Azure','histogram':'Histogram','google':'Google','chatgpt':'ChatGPT'}

color2id = {color:i for i,color in enumerate("red, pink, orange, yellow, purple, green, blue, brown, gray, black, white".split(', '))}
id2color = {i:color for i,color in enumerate("red, pink, orange, yellow, purple, green, blue, brown, gray, black, white".split(', '))}
# color2id['yellow'] = 2
count = 0
participant_count = 0
trials_dict = {}
all_data = cursor.fetchall()
completed_counterbalances = []


confmat_overall = construct_confmat()
catch_trial_mat_overall = np.zeros((1,2))
swap_trial_mat_overall = np.zeros((1,2))

color_tagging_dict = {}

for i in range(len(all_data)):
    if all_data[i][15] in [5]:
        if all_data[i][3] not in ["3ZQX1VYFUOCXJIZSTMP3ESBC33LO8M"]:#,"3FJ2RVH26ADJQGTXBGXK7B3KM4892R","3FJ2RVH26ADJQGTXBGXK7B3KM4A29M"]:
            continue
        data_dict = json.loads(all_data[i][17])
        if data_dict['mode'] != 'live':
            continue
        # filter out subjects using dummy trial accuracy
        # original threshold in the paper: 7/12
        dummy_mark = 0
        nTrials = 50
        threshold = nTrials*(7/12) # change to number of total trials 
        for dat in data_dict['data']:

            if dat['trialdata']['phase'] == 'TEST':
                present = dat['trialdata']['oneWord_curr'] == '0'
                response_present = dat['trialdata']['topic'] == '1'
                if present == response_present:
                    dummy_mark += 1
        if dummy_mark < threshold:
            continue
        
        
        # construct conf matrices
        time_used, counterbalance = print_metadata(all_data[i],"{}/{}".format(dummy_mark,nTrials))
        completed_counterbalances.append(counterbalance)
#         marks = []
#         marks_conf = []
        confmatrices = construct_confmat() # without catch
        catch_trial_mat = np.zeros((1,2))
        swap_trial_mat = np.zeros((1,2))
        marks = []
        marks_conf = []
        participant_count += 1
        nTrial = 0


        for dat in data_dict['data']:
            if dat['trialdata']['phase'] == 'TEST':
                
                nTrial+=1
                trial_data = {}
                trial_data['workerID'] = data_dict['workerId']
                trial_data['workerData'] = data_dict['questiondata']
                trial_data['trialData'] = dat['trialdata']

                print(count, trial_data['trialData']['imageID'], trial_data['trialData']['color'].lower())
                if trial_data['trialData']['color'].lower() == 'pink':
                    print('****')
                color_tagging_dict.setdefault(trial_data['trialData']['imageID'],[]).append(color2id[trial_data['trialData']['color']])
                count += 1
            
                trial_data['trialData']['color'] = trial_data['trialData']['color'].lower()
                del trial_data['trialData']['color']          

                trial_data['trialData']['response_object'] = trial_data['trialData']['topic'].lower()
                del trial_data['trialData']['topic']     
                
            
                trial_data['trialData']['object_groundtruth'] = trial_data['trialData']['oneWord_curr'].lower()
                del trial_data['trialData']['oneWord_curr']  
                trial_data['trialData']['imageID'] = trial_data['trialData']['imageID'].lower()
                del trial_data['trialData']['imageID']              
            
                trials_dict['Trial_ColorConst_'+str(count)] = trial_data

conn.close()

[Finished Date] 2025-05-26 06:04:14
[Time Used] 5.5 mins
[Questionnaire] {'country': 'United States', 'age': '31-35', 'engagement': '6', 'difficulty': '4', 'mode': 'live', 'gender': 'male', 'education': 'high school', 'ai-experience': '2', 'condition': 0, 'native': 'yes'}
[Counterbalance] 0
[Dummy Test Acc] 46/50
0 https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/568.jpg blue
1 https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/558.jpg brown
2 https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/552.jpg green
3 https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/405.jpg gray
4 https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/257.jpg black
5 https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/941.jpg black
6 https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/698.jpg black
7 https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/167.jpg green
8 https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/831.jpg gray
9 https://xiaoturing.s3.us-east-1.amazon

In [40]:
from statsmodels.stats.inter_rater import fleiss_kappa, aggregate_raters
# subjects_color_tagging = {}
# for img, items in color_tagging_dict.items():
#     for subj, color in enumerate(items):
#         subjects_color_tagging.setdefault(subj,[]).append(color)

rate_count = aggregate_raters(list(color_tagging_dict.values()))
fleiss_kappa(rate_count[0])

0.684623397312549

In [42]:
rate_count[1]

array([ 0,  2,  3,  5,  6,  7,  8,  9, 10])

In [43]:
for i, row in enumerate(rate_count[0]):
    if max(row)< 12:
       print(list(color_tagging_dict.keys())[i])
       # print(row)
       print(["{}: {}".format(id2color[rate_count[1][j]],cnt) for j,cnt in enumerate(row) if cnt != 0])
    elif max(row) == 20:
       print(list(color_tagging_dict.keys())[i])

       print(["{}: {}".format(id2color[rate_count[1][j]],cnt) for j,cnt in enumerate(row) if cnt != 0])
    if  max(row) >= 15:
        

https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/558.jpg
['orange: 2', 'yellow: 7', 'brown: 3', 'gray: 1', 'black: 1', 'white: 6']
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/405.jpg
['gray: 20']
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/941.jpg
['black: 20']
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/951.jpg
['green: 20']
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/1067.jpg
['gray: 20']
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/502.jpg
['gray: 20']
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/913.jpg
['green: 20']
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/842.jpg
['gray: 20']
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/108.jpg
['yellow: 8', 'green: 6', 'brown: 3', 'black: 2', 'white: 1']
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/427.jpg
['brown: 10', 'gray: 8', 'white: 2']
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/5.jpg
['brown: 20']
https://xiaoturi

In [53]:
import os
from collections import Counter
color_d = {}

ROOT = "/home/xiao/Downloads/Color_ChatGPT/Color_ChatGPT/static/responses_color_chatgpt/"
for fp in os.listdir(ROOT):
    with open(ROOT +fp,'r') as f:
        response = f.readlines()[0]
    color = response.split("Speaker Response: ")[1].split("</h4> </center>")[0]
    link = "https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/{}.jpg".format(fp.split('_')[0])
    model = {'a':'human','b':'azure','c':'histogram','d':'google','e':'chatgpt'}[fp.split('_')[1]]
    if model == "human":
        continue
    color_d.setdefault(link,[]).append([model,color.lower()])



for imageid, colors in color_d.items():
    color_d[imageid] = {k:v for k,v in colors}

for i, row in enumerate(rate_count[0]):
    if  max(row) >= 15:
        j = [j for j,cnt in enumerate(row) if cnt ==max(row)][0]
        clr = id2color[rate_count[1][j]]
        mid = list(color_tagging_dict.keys())[i]
        color_d[mid]['human'] = color
        
for imageid, colors_items in color_d.items():
    colors = list(colors_items.values())
    # if Counter(colors).most_common(1)[0][-1] == 5:
    #     consistents5.append(imageid)
    # elif Counter(colors).most_common(1)[0][-1] == 4:
    #     consistents4.append(imageid)
    if 'human' not in colors_items:
        continue
    model_cnt = Counter(list([v for k,v in colors_items.items() if k != 'human'])).most_common(1)[0]
    if model_cnt[-1] >=3 and model_cnt[0] != colors_items['human']:
        # if model_cnt[0] not in ['gray','black','white','brown'] and colors_items['human'] not in ['gray','black','white','brown']:
        print(imageid)
        print(colors_items)

https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/528.jpg
{'chatgpt': 'green', 'google': 'green', 'azure': 'green', 'histogram': 'green', 'human': 'brown'}
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/396.jpg
{'histogram': 'gray', 'chatgpt': 'gray', 'google': 'gray', 'azure': 'gray', 'human': 'brown'}
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/257.jpg
{'azure': 'black', 'chatgpt': 'black', 'google': 'black', 'histogram': 'black', 'human': 'brown'}
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/798.jpg
{'chatgpt': 'gray', 'histogram': 'gray', 'azure': 'gray', 'google': 'gray', 'human': 'brown'}
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/552.jpg
{'google': 'green', 'azure': 'green', 'histogram': 'green', 'chatgpt': 'green', 'human': 'brown'}
https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/15.jpg
{'azure': 'gray', 'chatgpt': 'gray', 'histogram': 'gray', 'google': 'gray', 'human': 'brown'}
https://xiaoturing.s3.us-east-1.amazonaws

In [49]:
color_d

{'https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/608.jpg': {'human': 'black',
  'histogram': 'green',
  'google': 'orange',
  'azure': 'black',
  'chatgpt': 'red'},
 'https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/921.jpg': {'chatgpt': 'gray',
  'histogram': 'gray',
  'human': 'brown',
  'azure': 'white',
  'google': 'green'},
 'https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/606.jpg': {'google': 'black',
  'histogram': 'green',
  'azure': 'gray',
  'human': 'gray',
  'chatgpt': 'black'},
 'https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/1065.jpg': {'chatgpt': 'green',
  'histogram': 'gray',
  'google': 'red',
  'azure': 'white',
  'human': 'gray'},
 'https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/1148.jpg': {'azure': 'gray',
  'chatgpt': 'gray',
  'google': 'gray',
  'histogram': 'gray',
  'human': 'pink'},
 'https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/478.jpg': {'azure': 'white',
  'human': 'red',
  'histogram': 'green',
  'chatg

In [23]:
with open("/home/liuxiao/TuringGithubRunnable/dominant_color_recognition/Plot/Data_all/humanjudge_Color_ChatGPT_testphase.json", 'w') as f:
    json.dump(trials_dict,f)

In [14]:
len(trials_dict)

2000

In [22]:
confmat_overall

{'h_machine': array([[545., 455.],
        [408., 592.]]),
 'h_azure': array([[ 93., 157.]]),
 'h_histogram': array([[ 75., 175.]]),
 'h_google': array([[ 85., 165.]]),
 'h_chatgpt': array([[155.,  95.]])}

In [19]:
trials_dict

{'Trial_Color_1': {'workerID': 'A1FVXS8IM5QYO8',
  'workerData': {'country': 'United Kingdom',
   'age': '51',
   'engagement': '10',
   'difficulty': '8',
   'mode': 'live',
   'gender': 'female',
   'education': 'high school',
   'ai-experience': '1',
   'condition': 0,
   'native': 'yes'},
  'trialData': {'rt': 14236,
   'Aclass': 'H',
   'hit': '1',
   'imageID': 'https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/553.jpg',
   'trial': 1,
   'page_name': '553_a_0.html/',
   'phase': 'TEST',
   'response_gender': 'NA',
   'response_speaker': 'h',
   'response_object': '2',
   'groundtruth': 'human',
   'object_groundtruth': '0',
   'machine_groundtruth': ''}},
 'Trial_Color_2': {'workerID': 'A1FVXS8IM5QYO8',
  'workerData': {'country': 'United Kingdom',
   'age': '51',
   'engagement': '10',
   'difficulty': '8',
   'mode': 'live',
   'gender': 'female',
   'education': 'high school',
   'ai-experience': '1',
   'condition': 0,
   'native': 'yes'},
  'trialData': {'rt': 10297,


In [78]:
len()

(1280,)

In [79]:
trial_data['trialData']

{'rt': 5129,
 'Aclass': 'Machine',
 'imageID': 'https://xiaoturing.s3.us-east-1.amazonaws.com/color_det/65.jpg',
 'trial': 100,
 'page_name': '65_d_0.html/',
 'phase': 'TEST',
 'response_gender': 'NA',
 'response_speaker': 'machine',
 'response_object': '1',
 'groundtruth': 'machine',
 'object_groundtruth': '0',
 'machine_groundtruth': 'Google',
 'imgtype': 'naturaldesign'}